In [7]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.mft import MFT
from checklist.inv_dir import INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.perturb import Perturb


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
model = model_wrapper.ModelWrapper()
new_pp = PredictorWrapper.wrap_softmax(model.predict_proba)


In [9]:
editor = checklist.editor.Editor()
editor.tg

In [10]:
nlp = spacy.load('en_core_web_sm')

In [11]:
qs = []
labels = []
all_questions = set()
for x in open('/home/marcotcr/datasets/glue/glue_data/QQP/dev.tsv').readlines()[1:]:
    try:
        q1, q2, label = x.strip().split('\t')[3:]
    except:
        print(x)
        continue
    all_questions.add(q1)
    all_questions.add(q2)
    qs.append((q1, q2))
    labels.append(label)
labels = np.array(labels).astype(int)

"	What does the Quran say about homosexuality?	0



In [12]:
import pickle
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_qqp.pkl', 'rb'))

In [13]:
parsed_qs = [(spacy_map[q[0]], spacy_map[q[1]]) for q in qs]

## Vocabulary

In [14]:
professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
print(', '.join(professions))

journalist, historian, nurse, engineer, accountant, attorney, architect, artist, editor, actor, actress, analyst, economist, entrepreneur, intern, author, investor, interpreter, assistant, executive, escort, organizer, educator, academic, agent, activist, advisor, administrator, investigator, auditor


In [15]:
professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:bert}.')[:30]
professions = list(set(professions))
# professions

In [16]:
print(', '.join(editor.suggest('{first_name} {last_name} is a good {bert}.')[:30]))

guy, player, writer, example, actor, man, hitter, person, friend, read, kid, shooter, coach, one, dude, pick, character, fighter, teacher, shot, artist, student, poet, reader, reporter, quarterback, catch, athlete, name, sport


In [17]:
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))

In [18]:
print(', '.join(editor.suggest('Is {first_name} {last_name} {a:bert} {noun}?', noun=nouns)[:50]))

effective, actual, active, important, ideal, excellent, American, outstanding, honest, acceptable, elite, unusual, interesting, average, appropriate, adequate, evil, international, experienced, amazing, official, independent, accurate, old, angel, established, English, exceptional, authentic, original, unlikely, Australian, accomplished, accidental, aspiring, intelligent, inexperienced, early, exemplary, ordinary, good, abusive, ethical, older, awesome, expert, eligible, LGBT, incompetent, acting


In [19]:
adjs = ['effective', 'actual', 'American', 'active', 'honest', 'excellent', 'elite', 'acomplished', 'official', 'outstanding', 'experienced', 'independent', 'international', 'aspiring', 'average', 'good', 'amazing', 'exceptional', 'successful', 'accredited', 'English', 'real', 'bad', 'terrible', 'fake', 'unusual', 'influential', 'incompetent']

In [13]:
data = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} {a:adj} {noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    766 (76.6%)

Example fails:
1.0 ('Is Shannon James an escort?', 'Is Shannon James an excellent escort?')
----
1.0 ('Is Kyle Gutierrez an activist?', 'Is Kyle Gutierrez an acomplished activist?')
----
1.0 ('Is Timothy Gonzalez a chef?', 'Is Timothy Gonzalez an average chef?')
----


Is person {a1, a2}?

In [14]:
print(', '.join(editor.suggest('Is John Wayne {bert}?')))
print()
print(', '.join(editor.suggest('Is John Wayne {a:bert}?')[:50]))

dead, gay, alive, right, Dead, real, Jewish, mad, wrong, correct, insane, back, crazy, related, OK, Right, immortal, evil, straight, interested, famous, racist, sane, next, President, alone, lying, relevant, okay, DEAD, bisexual, God, angry, guilty, ok, Alone, Wrong, ready, gone, innocent, King, human, Muslim, Black, assassinated, Batman, here, Real, dying, involved, homosexual, special, Famous, happy, free, murdered, Superman, president, joking, White, Back, Gay, Catholic, cool, safe, resurrected, killed, American, missing, corrupt, cursed, alright, single, Crazy, Mad, Satan, Missing, legit, shot, Evil, finished, worthy, doomed, sincere, aware, autistic, responsible, dangerous, Christian, Cool, Immortal, radioactive, remembered, Next, serious, cheating, different, popular, Legend, listening, now, illegitimate, homophobic, kidding, sick, smart, connected, trustworthy, Innocent, legal

racist, Christian, homosexual, atheist, rapist, murderer, Nazi, outlaw, asshole, actor, ape, American,

In [15]:
adjs_without_overlap = ['dead', 'gay', 'Jewish', 'Christian', 'American', 'mad', 'immortal', 'evil', 'famous', 'racist', 'Muslim', 'white', 'black', 'English', 'autistic', 'Australian', 'trustworthy', 'an atheist', 'an anarchist', 'an inventor', 'Indian', 'Armenian', 'an astronaut', 'an immigrant']

In [16]:
data = editor.template((
    'Is {first_name} {last_name} {adj1}?',
    'Is {first_name} {last_name} {adj2}?',
),
                adj=adjs_without_overlap,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 955 examples
Test cases:      955
Fails (rate):    5 (0.5%)

Example fails:
0.5 ('Is Jeffrey Harris Jewish?', 'Is Jeffrey Harris an immigrant?')
----
0.9 ('Is Taylor Rogers immortal?', 'Is Taylor Rogers dead?')
----
0.8 ('Is Richard Roberts Jewish?', 'Is Richard Roberts an immigrant?')
----


person1 and person2 are different by first and last name

In [17]:
data = editor.template((
    'Is {first_name1} {last_name1} {adj}?',
    'Is {first_name2} {last_name2} {adj}?',
),
                adj=adjs_without_overlap,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 971 examples
Test cases:      971
Fails (rate):    60 (6.2%)

Example fails:
1.0 ('Is Benjamin Jones white?', 'Is Noah Ward white?')
----
0.7 ('Is Madison Rodriguez immortal?', 'Is Shannon Hernandez immortal?')
----
0.8 ('Is Ethan Reyes evil?', 'Is Austin Russell evil?')
----


person1 and person2 are different by first name only

In [18]:
data = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name2} {last_name} {adj}?',
),
                adj=adjs_without_overlap,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 985 examples
Test cases:      985
Fails (rate):    94 (9.5%)

Example fails:
0.7 ('Is Lauren White an inventor?', 'Is Alyssa White an inventor?')
----
0.7 ('Is Christina Gomez Christian?', 'Is Melissa Gomez Christian?')
----
0.9 ('Is Jonathan Turner mad?', 'Is James Turner mad?')
----


person1 and person2 are different by last name only

In [19]:
data = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name} {last_name2} {adj}?',
),
                adj=adjs_without_overlap,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 991 examples
Test cases:      991
Fails (rate):    61 (6.2%)

Example fails:
0.8 ('Is Victoria Reyes English?', 'Is Victoria Ortiz English?')
----
0.9 ('Is Angela Davis mad?', 'Is Angela Taylor mad?')
----
0.8 ('Is Nicholas Jackson white?', 'Is Nicholas Adams white?')
----


## Robustness

In [20]:
def wrap_apply_to_each(fn, both=False, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, str(q2)) for x in fnq1])
        ret.extend([(str(q1), x) for x in fnq2])
        if both:
            ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn
def wrap_apply_to_both(fn, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn

Typos

In [21]:
data = Perturb.perturb(qs, wrap_apply_to_each(Perturb.add_typos), nsamples=500)
test = INV(data)
test.run(new_pp)
test.summary(3)

Predicting 1500 examples
Test cases:      500
Fails (rate):    120 (24.0%)

Example fails:
0.1 ('Is it a good idea to never open the lights in the evening to avoid blue light from interfering your sleep?', 'What evidence is there that exposure to blue light (say from a computer) late in the evening interferes with sleep?')
0.6 ('Is it a good idea to never open the lights in the evening to avoid blue light from interfering your sleep?', 'What evidence is tehre that exposure to blue light (say from a computer) late in the evening interferes with sleep?')

----
0.7 ('How can a woman rape a man?', 'Is it possible for a woman to rape a man?')
0.0 ('How can a woman rape a man?', 'Is it possible for aw oman to rape a man?')
0.0 ('How can  awoman rape a man?', 'Is it possible for a woman to rape a man?')

----
1.0 ('How many videos can YouTube hold?', 'How many videos are on YouTube?')
0.0 ('How many videos can YouTube hold?', 'How many videos are on YouTub?e')

----


Contractions

In [23]:
data = Perturb.perturb(qs, wrap_apply_to_each(Perturb.contractions, both=True), nsamples=500)
test = INV(data)
test.run(new_pp)
test.summary(3)

Predicting 1524 examples
Test cases:      500
Fails (rate):    15 (3.0%)

Example fails:
0.7 ('How does the AC Generator work?', 'How is AC electricity generated?')
0.1 ('How does the AC Generator work?', "How's AC electricity generated?")

----
0.8 ('Where is Chris Christie?', 'Where is Chris Christie from?')
0.0 ("Where's Chris Christie?", 'Where is Chris Christie from?')
0.3 ("Where's Chris Christie?", "Where's Chris Christie from?")

----
0.0 ('How did UChicago get so strong at Statistics?', 'How did Cornell get so strong in statistics?')
0.7 ("How'd UChicago get so strong at Statistics?", "How'd Cornell get so strong in statistics?")
0.6 ('How did UChicago get so strong at Statistics?', "How'd Cornell get so strong in statistics?")

----


## Ner

### Change same name, number, location in both

Names

In [20]:
def change_both_wrapper(fn):
    def change_both(qs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True)
        c2 = fn(q2, seed=seed, meta=True)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        return [(q1, q2) for q1, q2, m1, m2 in zip(c1, c2, m1, m2) if m1 == m2]
    return change_both
    
data = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_names), nsamples=500)
test = INV(data)
test.run(new_pp)
test.summary(3)

Predicting 5366 examples
Test cases:      500
Fails (rate):    58 (11.6%)

Example fails:
0.9 ('Is Hillary Clinton worried about the email scandals?', "Why isn't Hillary Clinton concerned about her email scandal?")
0.3 ('Is Emily Powell worried about the email scandals?', "Why isn't Emily Powell concerned about her email scandal?")
0.4 ('Is Ashley Jones worried about the email scandals?', "Why isn't Ashley Jones concerned about her email scandal?")

----
1.0 ('How will Donald Trump benefit India?', 'What Can happen to India if Donald Trump becomes president?')
0.0 ('How will John Perez benefit India?', 'What Can happen to India if John Perez becomes president?')
0.1 ('How will David Reyes benefit India?', 'What Can happen to India if David Reyes becomes president?')

----
1.0 ("Why isn't Hillary Clinton in jail?", 'Could Hillary Clinton actually go to jail?')
0.1 ("Why isn't Stephanie Ward in jail?", 'Could Stephanie Ward actually go to jail?')
0.3 ("Why isn't Melissa Nelson in jail?",

In [ ]:
test.summary(3)

Locs

In [25]:
data = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_location), nsamples=500)
test = INV(data)
test.run(new_pp)
test.summary(3)

Predicting 5109 examples
Test cases:      500
Fails (rate):    60 (12.0%)

Example fails:
0.0 ('What is the best way to tour/experience the north east of India in seven days?', 'Which are the 5 best places to visit in North-east India?')
1.0 ('What is the best way to tour/experience the north east of Kyrgyz Republic in seven days?', 'Which are the 5 best places to visit in North-east Kyrgyz Republic?')
0.8 ('What is the best way to tour/experience the north east of Kiribati in seven days?', 'Which are the 5 best places to visit in North-east Kiribati?')

----
0.4 ('What makes you happy about India?', 'What makes everyone happy about India?')
1.0 ('What makes you happy about Sint Maarten (Dutch part)?', 'What makes everyone happy about Sint Maarten (Dutch part)?')
1.0 ('What makes you happy about Virgin Islands (U.S.)?', 'What makes everyone happy about Virgin Islands (U.S.)?')

----
0.8 ('Where can adderalls be found in India, without prescription and without ADHD?', 'Can you get Adder

Numbers

In [26]:
data = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_number), nsamples=500)
test = INV(data)
test.run(new_pp)
test.summary(3)

Predicting 5025 examples
Test cases:      500
Fails (rate):    31 (6.2%)

Example fails:
0.6 ('WHEN will Redmi note 3 get Marshmallow?', 'Will Redmi Note 3 get Marshmallow Update?')
0.4 ('WHEN will Redmi note 4 get Marshmallow?', 'Will Redmi Note 4 get Marshmallow Update?')
0.4 ('WHEN will Redmi note 4 get Marshmallow?', 'Will Redmi Note 4 get Marshmallow Update?')

----
0.0 ('Honda CBR 250R vs KTM RC 390 vs Kawasaki Ninja 300. Which one should I buy in India?', 'Which bike should I buy: RC 390 or CBR 250?')
0.9 ('Honda CBR 250R vs KTM RC 406 vs Kawasaki Ninja 300. Which one should I buy in India?', 'Which bike should I buy: RC 406 or CBR 250?')

----
0.2 ('How do I get started for cds exam 2016?', 'How I can apply for cds exam 2016?')
0.7 ('How do I get started for cds exam 1945?', 'How I can apply for cds exam 1945?')
0.7 ('How do I get started for cds exam 1836?', 'How I can apply for cds exam 1836?')

----


### Change name, loc, number in only one where orig prediction is duplicate

Changing only first names

In [43]:
data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500, first_only=True)
expect_fn = Expect.eq(0)
expect_fn = Expect.slice_orig(expect_fn, lambda orig, *args: orig == 1)
test = DIR(data, expect_fn)
test.run(new_pp)
test.summary(3)


Predicting 23020 examples
Test cases:      1500
After filtering: 549 (36.6%)
Fails (rate):    488 (88.9%)

Example fails:
1.0 ('What are the best gaming laptops under INR 60000?', 'What are some of the best gaming laptops under rs 60000 that are available in india?')
1.0 ('What are the best gaming laptops under INR 58216?', 'What are some of the best gaming laptops under rs 60000 that are available in india?')
1.0 ('What are the best gaming laptops under INR 60000?', 'What are some of the best gaming laptops under rs 57094 that are available in india?')

----
1.0 ('How did the Mayans find out that the world was going to end in 2012 (completely ignoring the fact that they were wrong)?', '"Why did the Mayans think that the world ""is going to end"" in 2012?"')
1.0 ('How did the Mayans find out that the world was going to end in 2012 (completely ignoring the fact that they were wrong)?', '"Why did the Mayans think that the world ""is going to end"" in 2078?"')
1.0 ('How did the Mayans fin

Changing first and last names

In [36]:
data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500)
test = DIR(data, expect_fn)
test.run(new_pp)
test.summary(3)


Predicting 23570 examples
Test cases:      1500
After filtering: 601 (40.1%)
Fails (rate):    291 (48.4%)

Example fails:
1.0 ('How are the Indians affected if Donald Trump wins the presidential election in the US?', 'What might be the implications and repercussions on India if Donald Trump wins US Presidential elections?')
1.0 ('How are the Indians affected if Donald Trump wins the presidential election in the US?', 'What might be the implications and repercussions on India if Michael Adams wins US Presidential elections?')
1.0 ('How are the Indians affected if James Scott wins the presidential election in the US?', 'What might be the implications and repercussions on India if Donald Trump wins US Presidential elections?')

----
1.0 ('What are some good side dishes to serve with homemade Mac and cheese?', 'What are some great side dishes that go well with mac and cheese?')
0.9 ('What are some good side dishes to serve with homemade Patrick and cheese?', 'What are some great side dishe

Locs

In [37]:
data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_location), nsamples=1500)
test = DIR(data, expect_fn)
test.run(new_pp)
test.summary(3)

Predicting 23450 examples
Test cases:      1500
After filtering: 488 (32.5%)
Fails (rate):    225 (46.1%)

Example fails:
1.0 ('Is this move of banning 500 & 1000 Rupee notes right?', 'What will be the result of banning 500 and 1000 rupees note in India?')
1.0 ('Is this move of banning 500 & 1000 Rupee notes right?', 'What will be the result of banning 500 and 1000 rupees note in United Arab Emirates?')
1.0 ('Is this move of banning 500 & 1000 Rupee notes right?', 'What will be the result of banning 500 and 1000 rupees note in Bahamas?')

----
1.0 ('How much do you pay to driver in Pune, India?', 'How much do you pay to a driver in Pune, India?')
1.0 ('How much do you pay to driver in Pune, India?', 'How much do you pay to a driver in Pune, Cayman Islands?')
1.0 ('How much do you pay to driver in Pune, India?', 'How much do you pay to a driver in Pune, United States?')

----
1.0 ('Is Spotify available in India?', 'When will Spotify arrive in India?')
1.0 ('Is Spotify available in Egypt

numbers

In [38]:

data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_number), nsamples=1500)
test = DIR(data, expect_fn)
test.run(new_pp)
test.summary(3)

Predicting 23020 examples
Test cases:      1500
After filtering: 549 (36.6%)
Fails (rate):    488 (88.9%)

Example fails:
1.0 ('What are the best bike technologies in 2016?', 'What are the best bike inventions of 2016?')
1.0 ('What are the best bike technologies in 2016?', 'What are the best bike inventions of 2043?')

----
1.0 ("India's Prime Minister removed 500 and 1000 rupee notes from circulation. Is this a good way to curb the spread of black money?", 'Why is 500 and 1000 rupee notes discontinued?')
1.0 ("India's Prime Minister removed 500 and 1000 rupee notes from circulation. Is this a good way to curb the spread of black money?", 'Why is 500 and 927 rupee notes discontinued?')
1.0 ("India's Prime Minister removed 500 and 1000 rupee notes from circulation. Is this a good way to curb the spread of black money?", 'Why is 509 and 1000 rupee notes discontinued?')

----
1.0 ('What are the effects of demonitization of 500 and 1000 rupees notes on real estate sector?', 'What will be t

In [39]:
### Change 

### Keep entities, fill in with BERT gibberish

In [53]:
def bert_gibberish(question):
    ents = question.ents
    if not ents:
        return None
    wp = [x.text for x in question if x.tag_ in ['WP', 'WRB', 'WDT']]
    if not wp:
        wp = question[0].text
    else:
        wp = wp[0]
    ents = [x.text for x in ents]
    ents[-1] = ents[-1] + '?'
    template = ' {bert} '.join([wp] + ents)
    gibberish = editor.template(template)[:5]
#     return gibberish
    ret = [(question.text, x) for x in gibberish if question.text != x ]
    return ret
def gibberish_both(qs):
    q1, q2 = qs
    ret = []
    x1 = bert_gibberish(q1)
    if x1:
        ret.extend(x1)
    x2 = bert_gibberish(q2)
    if x2:
        ret.extend(x2)
    return ret
#     ret =  [(question.text, x[1]) for x in tg.fill_in_between([wp] + ents) if question.text != x[1]]
#     ret = [x for x in ret if len(x[1].split()) < len(x[0].split()) -4][:5]
#     return ret 

In [48]:
data = Perturb.perturb(parsed_qs, gibberish_both, nsamples=500)
expect_false = Expect.eq(0)
test = DIR(data, expect_false)
test.run(new_pp)
test.summary(3)

Test cases:      500
Fails (rate):    166 (33.2%)

Example fails:
0.0 ('What is your review of Samsung Galaxy?', 'What is your review of Samsung Galaxy S6?')
1.0 ('What is your review of Samsung Galaxy?', 'What About Samsung Galaxy?')
1.0 ('What is your review of Samsung Galaxy?', 'What about Samsung Galaxy?')

----
0.0 ('What is Clayton Kershaw like in person?', "What is Clayton Kershaw's salary?")
0.9 ("What is Clayton Kershaw's salary?", "What 's Clayton Kershaw's?")
0.9 ("What is Clayton Kershaw's salary?", "What Is Clayton Kershaw's?")

----
0.0 ("What's your JEE Mains score?", 'What is JEE Main ?')
1.0 ('What is JEE Main ?', "What 's JEE Main?")
1.0 ('What is JEE Main ?', 'What are JEE Main?')

----


In [54]:
test.summary(3)

Test cases:      500
Fails (rate):    166 (33.2%)

Example fails:
0.1 ('What is the cut off in coep for mechanical girls?', 'What is the cut-off for COEP?')
1.0 ('What is the cut-off for COEP?', 'What about COEP?')
1.0 ('What is the cut-off for COEP?', 'What About COEP?')

----
1.0 ('How can you get the best grades at school?', 'What can I do to get better grades next quarter?')
1.0 ('What can I do to get better grades next quarter?', 'What for next quarter?')
0.5 ('What can I do to get better grades next quarter?', 'What happen next quarter?')

----
1.0 ('Why did AT&T purchase Time Warner?', 'Why is AT&T buying Time Warner?')
1.0 ('Why did AT&T purchase Time Warner?', 'Why sold AT&T to Time Warner?')
0.9 ('Why is AT&T buying Time Warner?', 'Why both AT&T and Time Warner?')

----


## Temporal

Is != used to be

In [55]:
data = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Did {first_name} {last_name} use to be {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    737 (73.7%)

Example fails:
0.9 ('Is Victoria Foster a chef?', 'Did Victoria Foster use to be a chef?')
----
0.6 ('Is Ryan Adams an advisor?', 'Did Ryan Adams use to be an advisor?')
----
0.7 ('Is Mark James a mechanic?', 'Did Mark James use to be a mechanic?')
----


Is != becoming

In [56]:
data = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} becoming {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 999 examples
Test cases:      999
Fails (rate):    182 (18.2%)

Example fails:
0.8 ('Is Dylan Bailey a nurse?', 'Is Dylan Bailey becoming a nurse?')
----
0.7 ('Is Taylor King an employee?', 'Is Taylor King becoming an employee?')
----
0.8 ('Is Kayla Bailey a DJ?', 'Is Kayla Bailey becoming a DJ?')
----


Before != after

In [57]:
data = editor.template((
    'What was {first_name} {last_name}\'s life before becoming {a:noun}?',
    'What was {first_name} {last_name}\'s life after becoming {a:noun}?'
),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 999 examples
Test cases:      999
Fails (rate):    999 (100.0%)

Example fails:
1.0 ("What was Laura Green's life before becoming a writer?", "What was Laura Green's life after becoming a writer?")
----
1.0 ("What was Lauren Smith's life before becoming an entrepreneur?", "What was Lauren Smith's life after becoming an entrepreneur?")
----
1.0 ("What was Jacob Thompson's life before becoming an accountant?", "What was Jacob Thompson's life after becoming an accountant?")
----


In [58]:
print(', '.join(editor.suggest('Is is {bert} to eat after 10pm?')))

it, normal, OK, there, safe, best, ok, okay, acceptable, reasonable, permissible, legal, proper, wrong, healthy, good, enough, better, important, appropriate, left, allowed, possible, dinner, vegetarian, what, recommended, going, food, illegal, supposed, smart, unhealthy, hard, popular, hot, necessary, vegan, typical, not, free, right, required, fun, time, meant, something, one, kosher, customary, fair, mandatory, available, common, cheaper, alright, easier, suitable, feasible, polite, difficult, cool, certain, sensible, affordable, advisable, fine, fashionable, like, nice, unsafe, public, wise, decent, sufficient, harder, advised, traditional, usual, healthier, anyone, permitted, eating, easy, life, trendy, someone, forbidden, lawful, expensive, safer, new, everyone, compulsory, dangerous, worse, needed, realistic, temptation, essential, adequate, impossible, hungry, rude, risky, considered, prudent, taboo, different, tempting


In [59]:
mid = ['normal', 'ok', 'safe', 'dangerous', 'acceptable', 'reasonable', 'proper', 'wrong', 'healthy', 'important']

In [60]:
print(', '.join(editor.suggest('Is is {mid} to {bert} after 10pm?', mid=mid)))

sleep, drink, drive, eat, work, smoke, go, leave, travel, stay, walk, do, visit, read, be, exercise, watch, fish, talk, vote, play, call, party, move, retire, write, swim, run, study, live, post, celebrate, stop, text, shop, continue, relax, dance, return, start, tweet, disappear, act, speak, strike, die, fly, gamble, come, report, bed, change, meet, remain, enter, attend, kill, feed, see, gather, avoid, operate, park, cook, buy, breathe, film, have, wait, cycle, cross, paint, phone, check, spend, practice, pee, look, sit, commute, consume, skate, linger, use, pass, ride, bike, bet, wander, finish, blog, vape, approach, rise, roam, proceed, dive, stand, bicycle, say, explore, hike, stroll, hang, take, car, know, open, rest, disturb, emerge, escape, hunt, ask, exit, answer, follow, quit, people, crash, think, pray, dress, ski, wear, dinner, chat, cry, fight, close, exist, surf, evacuate, perform, vanish, arrive, garden, crawl, jump, jog, propose, children, behave, get, step, taxi, obser

In [61]:
activity = ['drink', 'sleep', 'drive', 'work', 'eat', 'smoke', 'walk', 'read', 'party', 'talk', 'exercise', 'celebrate', 'text', 'tweet', 'run', 'dance', 'swim', 'cook', 'pray', 'pee', 'rest']

In [62]:
data = editor.template(('Is it {mid} to {activity} before {hour}{ampm}?','Is it {mid} to {activity} after {hour}{ampm}?'),
                activity=activity,
                mid=mid,
                hour=[str(x) for x in range(1, 12)],
                ampm=['am', 'pm'],
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    976 (97.6%)

Example fails:
0.9 ('Is it proper to exercise before 6pm?', 'Is it proper to exercise after 6pm?')
----
0.9 ('Is it healthy to text before 3pm?', 'Is it healthy to text after 3pm?')
----
0.8 ('Is it healthy to read before 5pm?', 'Is it healthy to read after 5pm?')
----


## Negation

In [63]:
mid2 = mid + ['legal', 'awkward', 'socially acceptable']

In [64]:
print(', '.join(editor.suggest('Is it {mid} to {bert} in {country}?', mid=mid2)[:100]))

live, work, be, vote, drive, travel, marry, stay, study, fight, die, exist, speak, eat, campaign, fly, squat, pray, hunt, fish, visit, protest, remain, serve, write, smoke, shop, reside, settle, swim, move, play, meet, gamble, demonstrate, pee, operate, perform, kill, invest, surf, strike, train, march, report, arrive, gather, act, teach, murder, sleep, race, drink, hide, land, practice, vacation, spy, shoot, cook, fire, dance, survive, camp, migrate, holiday, worship, paint, burn, ski, rape, party, talk, walk, believe, exercise, barbecue, think, build, volunteer, propose, breed, ask, wait, lie, farm, preach, buy, stand, sue, start, gay, date, tan, rule, engage, appear, look, grow, park


In [65]:
things = ['work', 'vote', 'travel', 'marry', 'drive', 'study', 'protest', 'campaign', 'fight', 'gamble', 'hunt', 'pray', 'smoke', 'fish', 'murder', 'invest', 'pee', 'march', 'worship', 'rape', 'volunteer', 'surf', 'shoot', 'dance', 'camp', 'preach', 'spy', 'be gay', 'lie', 'divorce', 'discriminate']

In [66]:
data = editor.template(('Is it {mid} to {activity} in {country}?','Is it {mid} not to {activity} in {country}?'),
                activity=things,
                mid=mid2,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    223 (22.3%)

Example fails:
0.7 ('Is it wrong to marry in Liechtenstein?', 'Is it wrong not to marry in Liechtenstein?')
----
0.8 ('Is it wrong to fight in Serbia?', 'Is it wrong not to fight in Serbia?')
----
0.6 ('Is it dangerous to spy in Croatia?', 'Is it dangerous not to spy in Croatia?')
----


In [67]:
data = editor.template((
    'What are things {a:noun} should worry about?',
    'What are things {a:noun} should not worry about?',
),
                noun=nouns,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)


## Coref

In [68]:
data = editor.template(
    (
        'If {male} and {female} were alone, do you think he would reject her?',
        'If {male} and {female} were alone, do you think she would reject him?',
    ),
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    712 (71.2%)

Example fails:
0.8 ('If Jesse and Monica were alone, do you think he would reject her?', 'If Jesse and Monica were alone, do you think she would reject him?')
----
0.6 ('If Gabriel and Nicole were alone, do you think he would reject her?', 'If Gabriel and Nicole were alone, do you think she would reject him?')
----
0.6 ('If Jeremy and Abigail were alone, do you think he would reject her?', 'If Jeremy and Abigail were alone, do you think she would reject him?')
----


In [69]:
data = editor.template(
    [(
        'If {male} and {female} were married, would his family be happy?',
        'If {male} and {female} were married, would {female}\'s family be happy?',
    ),(
        'If {male} and {female} were married, would her family be happy?',
        'If {male} and {female} were married, would {male}\'s family be happy?',
    ),
    ]
        ,
    unroll=True,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 2000 examples
Test cases:      2000
Fails (rate):    1984 (99.2%)

Example fails:
0.9 ('If Anthony and Kaitlyn were married, would her family be happy?', "If Anthony and Kaitlyn were married, would Anthony's family be happy?")
----
0.9 ('If Alexander and Jessica were married, would her family be happy?', "If Alexander and Jessica were married, would Alexander's family be happy?")
----
0.9 ('If Kevin and Erica were married, would her family be happy?', "If Kevin and Erica were married, would Kevin's family be happy?")
----


## SRL

In [70]:
print(', '.join(editor.suggest('Who is the best {bert} in the world?')))

chef, pizza, boxer, player, footballer, athlete, rapper, actor, singer, cook, magician, robot, coach, beer, cyclist, wrestler, drummer, musician, quarterback, hacker, dog, baker, fighter, journalist, restaurant, steak, teacher, doctor, gamer, band, shooter, DJ, football, person, horse, photographer, driver, fisherman, burger, coffee, lawyer, writer, food, dancer, student, wine, artist, man, surgeon, comedian, trainer, VPN, vegetarian, programmer, team, game, vegan, guitarist, goalkeeper, server, guy, cricket, sport, engineer, dentist, waiter, cheese, bartender, fish, chicken, goalie, car, manager, computer, referee, hunter, guitar, basketball, tennis, astronaut, mathematician, sandwich, blogger, accountant, TV, soldier, sniper, job, runner, barbecue, chocolate, scientist, friend, cat, gun, pitcher, judge, AI, philosopher, butcher, painter, golf, camel, translator, piano, worker, farmer, conductor, politician, bike


In [71]:
thing = ['chef', 'boxer', 'player', 'footballer', 'athlete', 'rapper', 'actor', 'singer', 'cook', 'magician', 'coach', 'cyclist', 'wrestler', 'drummer', 'musician', 'quarterback', 'hacker', 'baker', 'fighter', 'journalist', 'teacher', 'doctor', 'gamer', 'husband', 'DJ', 'person', 'man', 'woman', 'surgeon', 'comedian', 'trainer', 'programmer', 'guitarist', 'goalkeeper']

In [72]:
print(', '.join(editor.suggest('Who do {bert} think is the the best {thing} in the world?', thing=thing)))

you, You, YOU, people, we, they, I, readers, u, guys, fans, your, others, ya, experts, Americans, scientists, men, voters, some, students, authors, the, analysts, everyone, conservatives, women, friends, Canadians, critics, he, judges, players, most, journalists, supporters, all, folks, audiences, liberals, historians, i, members, Australians, researchers, viewers, many, celebrities, artists, coaches, editors, respondents, participants, Republicans, writers, U, reporters, comedians, parents, teachers, economists, veterans, gamers, consumers, candidates, anyone, independents, kids, users, doctors, pundits, individuals, commentators, yo, politicians, ye, philosophers, millennials, reviewers, each, athletes, not, agents, feminists, investigators, psychologists, scholars, atheists, leaders, Christians, YOUR, archaeologists, both, police, children, ladies, stars, humans, still, those, pros, seniors, two, Democrats, adults, contestants, teenagers, any, listeners, academics, astronomers, Cons

In [73]:
subjects = ['you', 'people', 'readers', 'guys', 'fans', 'experts', 'scientists', 'Americans', 'students', 'men', 'voters', 'authors', 'conservatives', 'women', 'Canadians', 'analysts', 'critics', 'judges', 'artists', 'researchers', 'liberals', 'historians', 'Australians', 'journalists', 'Republicans', 'coaches', 'parents', 'kids', 'economists', 'reporters', 'consumers', 'veterans', 'doctors']

In [74]:
print(', '.join(editor.suggest('Who do {subjects} think is the the {bert} {thing} in the world?', thing=thing, subjects=subjects)[:50]))

best, greatest, smartest, worst, strongest, fastest, top, toughest, biggest, finest, happiest, deadliest, coolest, oldest, hottest, hardest, richest, safest, better, leading, great, brightest, largest, busiest, youngest, superior, Greatest, dominant, highest, most, quickest, easiest, favorite, tallest, ultimate, perfect, weakest, favourite, smallest, next, newest, wealthiest, first, Best, heaviest, only, right, simplest, elite, premier


In [75]:
best = ['best', 'greatest', 'worst', 'top', 'smartest', 'strongest', 'finests', 'happiest', 'coolest', 'richest', 'leading', 'brightest', 'premier', 'ultimate', 'dominant']

In [76]:
data = editor.template((
    'Who do {subjects} think is the {best} {thing} in the world?',
    'Who is the {best} {thing} in the world according to {subjects}?'
),
    subjects=subjects,
    best=best,
    thing=thing,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    130 (13.0%)

Example fails:
0.0 ('Who do women think is the finests athlete in the world?', 'Who is the finests athlete in the world according to women?')
----
0.4 ('Who do kids think is the leading cyclist in the world?', 'Who is the leading cyclist in the world according to kids?')
----
0.4 ('Who do consumers think is the worst actor in the world?', 'Who is the worst actor in the world according to consumers?')
----


In [77]:
print(', '.join([str(x) for x in editor.suggest('Are {bert} smaller than {a}?', a=['bananas', 'dogs', 'cars', 'cats', 'elephants'])][:100]))

cats, humans, dogs, people, pigs, chickens, you, cows, sheep, birds, mice, robots, fish, bees, snakes, bats, insects, trees, we, elephants, babies, puppies, animals, ants, they, spiders, butterflies, dolphins, cars, mosquitoes, plants, whales, dinosaurs, horses, goats, pets, men, computers, toys, things, flies, girls, rats, bears, monkeys, rabbits, houses, children, lions, bicycles, coins, kittens, drones, frogs, kids, ducks, turtles, bugs, worms, crabs, dolls, sharks, trucks, diamonds, tigers, wolves, bikes, beetles, primates, cattle, satellites, flowers, chimpanzees, carrots, ticks, apes, helicopters, cameras, mammals, reptiles, deer, ponies, rodents, stones, balls, seals, brains, seeds, rockets, boys, dragons, us, knives, cells, objects, women, balloons, pedestrians, bulls, apples


In [78]:
things = editor.suggest('Are {bert} smaller than {a}?',a=['bananas', 'dogs', 'cars', 'cats', 'elephants'] )[:100]

In [79]:
print(', '.join([str(x) for x in editor.suggest('Are {a} {bert} than {a2}?', a=things, a2=things)][:100]))

smarter, better, bigger, faster, stronger, worse, cooler, different, larger, smaller, more, tougher, safer, wiser, nicer, greater, happier, cleaner, less, weaker, healthier, slower, cheaper, other, quieter, quicker, taller, heavier, harder, lighter, brighter, older, hotter, easier, higher, closer, darker, louder, longer, colder, shorter, warmer, lower, simpler, intelligent, deeper, sharper, lesser, thicker, younger, wider, richer, stranger, important, broader, clearer, dangerous, superior, softer, rather, fewer, interesting, differently, smart, thinner, real, poorer, farther, inferior, tighter, valuable, dumb, finer, stupid, related, smoother, wealthier, er, anymore, sooner, strange, beautiful, stricter, ier, earlier, powerful, normal, scary, anything, further, weird, human, good, poisonous, rare, useful, male, crazy, true, funny


In [80]:
comp = ['better', 'worse', 'cheaper', 'bigger', 'louder', 'longer', 'larger', 'smaller', 'warmer', 'colder', 'thicker', 'lighter', 'heavier']

Order doesn't matter for comparison

In [81]:
data = editor.template((
    'Are {t1} {comp} than {t2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'What is {comp}, {t2} or {t1}?'
),
    t = things,
    comp = comp,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 991 examples
Test cases:      991
Fails (rate):    987 (99.6%)

Example fails:
0.0 ('Are toys bigger than apples?', 'What is bigger, apples or toys?')
----
0.0 ('Are dolphins larger than stones?', 'What is larger, stones or dolphins?')
----
0.0 ('Are deer colder than reptiles?', 'What is colder, reptiles or deer?')
----


Order matters for asymetric relations

In [82]:
# editor.template('Is {first_name1} {bert} {first_name2}?', remove_duplicates=True)[:100]

In [83]:
print(', '.join(editor.suggest('Is {first_name1} {bert} to {first_name2}?', remove_duplicates=True)[:100]))
print()
print(', '.join(editor.suggest('Is {first_name1} {bert} {first_name2}?', remove_duplicates=True)[:100]))

related, married, close, attracted, talking, engaged, connected, lying, closer, linked, speaking, loyal, going, important, mean, proposing, committed, listening, referring, up, faithful, responding, similar, on, next, true, dead, attached, writing, good, gay, dangerous, right, tied, allergic, returning, hostile, turning, Married, crazy, coming, bound, talk, lied, special, new, available, opposed, indebted, abusive, proposed, nice, known, attractive, real, closest, straight, back, out, addicted, kind, getting, relevant, friendly, reacting, truthful, nicer, alive, drawn, fair, happy, open, cruel, supposed, entitled, decent, devoted, guilty, happening, unfair, single, off, kin, Loyal, cold, bisexual, serious, acceptable, used, weird, gone, Close, down, insane, wed, pregnant, father, wrong, interesting, headed

or, really, and, an, a, with, Really, still, actually, like, /, dating, &, not, ,, from, real, after, born, the, …, even, indeed, now, married, called, marrying, his, truly, to, REA

In [84]:
symmetric = ['dating', 'married to', 'close to', 'engaged to', 'connected to', 'married to', 'friends with', 'related to', 'an acquaintance of']

In [85]:
data = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = symmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 994 examples
Test cases:      994
Fails (rate):    763 (76.8%)

Example fails:
0.0 ('Is Zachary married to Mark?', 'Is Mark married to Zachary?')
----
0.0 ('Is Dylan engaged to Natalie?', 'Is Natalie engaged to Dylan?')
----
0.0 ('Is Austin engaged to Nicole?', 'Is Nicole engaged to Austin?')
----


In [86]:
asymmetric = ['hurting', 'lying to', 'loyal to', 'faithful to', 'proposing to', 'indebted to', 'abusive to', 'using', 'expecting', 'beating', 'punching', 'raising', 'poisoning', 'protecting', 'kidnapping']

In [87]:
data = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = asymmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 994 examples
Test cases:      994
Fails (rate):    238 (23.9%)

Example fails:
0.6 ('Is Joseph poisoning Abigail?', 'Is Abigail poisoning Joseph?')
----
0.6 ('Is Shannon poisoning Tyler?', 'Is Tyler poisoning Shannon?')
----
0.5 ('Is Jason lying to Charles?', 'Is Charles lying to Jason?')
----


More traditional SRL, now possible with new interface (was hard before)

In [88]:
print(', '.join(editor.suggest('Did John buy the {bert}?', remove_duplicates=True)[:100]))

farm, stake, house, property, company, land, rights, ticket, papers, newspaper, book, island, estate, ranch, boat, horse, paper, shares, tickets, business, gun, books, newspapers, yacht, team, ship, stock, idea, plot, contract, Bible, phone, franchise, shotgun, store, horses, cattle, church, place, game, factory, castle, club, painting, bank, tract, rifle, manuscript, car, campaign, Ark, slaves, plane, school, money, guns, sword, plantation, beer, trust, election, film, ring, building, station, twins, deal, loan, time, tractor, home, letter, IRA, plan, title, watch, story, letters, rest, cows, piece, diary, brewery, charter, diamond, mortgage, hospital, site, post, operation, insurance, railroad, cow, coins, Dell, case, children, wine, telephone, dog


In [89]:
obj = ['farm', 'house', 'property', 'company', 'land', 'ticket', 'newspaper', 'book', 'island', 'estate', 'ranch', 'boat', 'horse', 'paper', 'business', 'gun', 'game', 'factory', 'castle', 'painting', 'rifle', 'car', 'school', 'building']

In [90]:
print(', '.join(editor.suggest('Did John {bert} the {obj}?', obj=obj, remove_duplicates=True)[:100]))

buy, sell, own, get, take, leave, lose, have, see, find, purchase, keep, want, manage, inherit, control, acquire, win, steal, miss, abandon, run, destroy, hold, handle, need, change, build, like, use, remember, bought, save, know, quit, sold, finish, enter, receive, seize, call, break, close, survive, mention, afford, fix, start, sign, enjoy, notice, rebuild, touch, accept, sue, left, join, give, maintain, fire, drop, move, retain, operate, deliver, open, raid, lead, found, attack, exit, carry, secure, rob, continue, burn, oversee, return, support, purchased, stop, obtain, organize, clear, split, develop, watch, read, improve, inherited, reopen, flee, complete, ruin, shut, choose, mind, attend, write, list


In [91]:
import pattern
import pattern.en
verbs = ['buy', 'purchase', 'sell', 'leave', 'own', 'take', 'keep', 'want', 'lose', 'destroy', 'inherit', 'find', 'use', 'need', 'receive', 'return', 'like', 'enjoy', 'abandon', 'manage', 'remember', 'miss', 'move', 'seize', 'steal']
a = pattern.en.tenses('stolen')[0]
verbs = [(v, pattern.en.conjugate(v, *a)) for v in verbs]
verbs[3] = ('leave', 'left')
verbs

[('buy', 'bought'),
 ('purchase', 'purchased'),
 ('sell', 'sold'),
 ('leave', 'left'),
 ('own', 'owned'),
 ('take', 'taken'),
 ('keep', 'kept'),
 ('want', 'wanted'),
 ('lose', 'lost'),
 ('destroy', 'destroyed'),
 ('inherit', 'inherited'),
 ('find', 'found'),
 ('use', 'used'),
 ('need', 'needed'),
 ('receive', 'received'),
 ('return', 'returned'),
 ('like', 'liked'),
 ('enjoy', 'enjoyed'),
 ('abandon', 'abandoned'),
 ('manage', 'managed'),
 ('remember', 'remembered'),
 ('miss', 'missed'),
 ('move', 'moved'),
 ('seize', 'seized'),
 ('steal', 'stolen')]

In [92]:
data = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was the {obj} {verb[1]} by {first_name}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    107 (10.7%)

Example fails:
0.2 ('Did Shannon miss the rifle?', 'Was the rifle missed by Shannon?')
----
0.3 ('Did John enjoy the painting?', 'Was the painting enjoyed by John?')
----
0.4 ('Did Kevin leave the ticket?', 'Was the ticket left by Kevin?')
----


In [93]:
data = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was {first_name} {verb[1]} by the {obj}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    944 (94.4%)

Example fails:
0.9 ('Did Sara lose the island?', 'Was Sara lost by the island?')
----
1.0 ('Did Nathan use the ticket?', 'Was Nathan used by the ticket?')
----
1.0 ('Did Adam buy the boat?', 'Was Adam bought by the boat?')
----


With people

In [94]:
print(', '.join(editor.suggest('Does {first_name} {bert} {first_name2}?', remove_duplicates=True)[:100]))

know, love, marry, hate, kill, like, want, remember, and, have, see, or, find, meet, choose, Know, date, miss, need, recognize, tell, Love, deserve, understand, prefer, leave, get, mention, trust, blame, forgive, support, murder, kidnap, accept, follow, mean, beat, survive, adopt, kiss, reject, take, hurt, notice, own, /, ask, save, dislike, resemble, visit, become, bother, recognise, Want, Remember, believe, call, ,, pursue, hit, ?, attack, rape, be, help, Like, contact, use, confront, adore, married, catch, fear, divorce, crush, clone, pick, invite, respect, suspect, regret, despise, Hate, stalk, eat, hire, KILL, killed, include, betray, resent, touch, &, protect, rescue, shoot, WANT, wed


In [95]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
data = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name2} {verb[1]} by {first_name}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(data, labels=1)
test.run(new_pp)
test.summary(n=3)

Predicting 994 examples
Test cases:      994
Fails (rate):    665 (66.9%)

Example fails:
0.2 ('Does Jonathan attack Jennifer?', 'Is Jennifer attacked by Jonathan?')
----
0.0 ('Does Mark hate David?', 'Is David hated by Mark?')
----
0.2 ('Does Nicole blame Sara?', 'Is Sara blamed by Nicole?')
----


In [96]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
data = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name} {verb[1]} by {first_name2}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 993 examples
Test cases:      993
Fails (rate):    973 (98.0%)

Example fails:
0.9 ('Does Brittany recognize Sophia?', 'Is Brittany recognized by Sophia?')
----
0.9 ('Does Jason hurt Samantha?', 'Is Jason hurt by Samantha?')
----
0.9 ('Does Andrea remember Anthony?', 'Is Andrea remembered by Anthony?')
----
